# Amazon Bedrock boto3 Setup

> *이 노트북은 SageMaker Studio의 **`Data Science 3.0`** 커널과 잘 작동합니다.*

---

이 데모 노트북에서는 [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)를 사용하여 [Amazon Bedrock](https://aws.amazon.com/bedrock/) 파운데이션 모델과 작업하는 방법을 보여줍니다.

---

## Prerequisites

이 섹션의 셀을 실행하여 Bedrock 워크샵에 필요한 패키지를 설치하세요. (Stable Diffusion 실습 뿐만 아니라 LLM 모델에 대한 다양한 실습을 위한 패키지를 설치합니다.)

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"


이 노트북에서는 AWS SDK를 사용하여 직접 Bedrock 모델을 호출하는 방법을 보여드리지만, 이후 워크샵의 노트북에서는 다른 패키지도 설치해야 합니다.

익숙하지 않을 수 있는 일부 패키지의 링크는 다음과 같습니다:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [sqlalchemy](https://www.sqlalchemy.org/), SQLAlchemy is the Python SQL toolkit 
- [Pinecone](http://pinecone.io), to store vector embeddings
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), for interactive UI widgets in the notebook
- [NeMo-Guardrails](https://github.com/NVIDIA/NeMo-Guardrails) an open-source toolkit for easily adding programmable guardrails to LLM-based conversational systems


### text

In [ ]:
%pip install --quiet \
    langchain==0.0.309 \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client \
    apache-beam \
    datasets \
    tiktoken \
    "ipywidgets>=7,<8" \
    matplotlib


### agents

In [ ]:
%pip install --quiet \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    pysqlite3 \
    google-search-results


### entity extraction

In [ ]:
%pip install --quiet beautifulsoup4


### image

In [ ]:
%pip install --quiet "pillow>=9.5,<10"


### guardrails

In [ ]:
%%bash
apt-get update && apt-get install g++ -y


In [ ]:

%pip install -qU --no-cache-dir nemoguardrails==0.5.0

%pip install -qU "faiss-cpu>=1.7,<2" \
                      "langchain==0.0.308" \
                      "pypdf>=3.8,<4" \
                      "ipywidgets>=7,<8"


---

## boto3 client 만들기

Bedrock API와의 상호 작용은 Python용 AWS SDK를 통해 이루어집니다: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

사용 중인 환경에 따라 Bedrock 서비스 클라이언트를 생성할 때 설정을 사용자 지정해야 할 수도 있습니다. 이를 돕기 위해 다양한 옵션 전달을 지원하는 `get_bedrock_client()` 유틸리티 메서드를 제공했습니다. 구현은 [../utils/bedrock.py](../utils/bedrock.py)에서 찾을 수 있습니다.

#### 다른 클라이언트 사용
boto3는 아마존 베드락이 다양한 작업을 수행할 수 있도록 다양한 클라이언트를 제공합니다. [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) 및 [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html)에 대한 작업은 Amazon Bedrock 런타임에서 지원되며, [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html)와 같은 다른 작업은 [Amazon Bedrock 클라이언트](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html)를 통해 처리됩니다.

`get_bedrock_client()` 메서드는 `runtime`(default=True) 매개변수를 받아 `bedrock` 또는 `bedrock-runtime` 클라이언트를 반환합니다.

#### 기본 자격증명 체인 사용

[Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/)에서 이 노트북을 실행하고 있고 Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)에 Bedrock에 액세스할 수 있는 권한이 있는 경우, 아래 셀을 그대로 실행하면 됩니다. 기본 AWS 자격 증명으로 Bedrock에 액세스할 수 있는 컴퓨터에서 이 노트북을 실행하는 경우에도 마찬가지입니다.

#### 다른 AWS Region 사용

자신의 컴퓨터에서 이 노트북을 실행하거나 Bedrock이 설정된 곳과 다른 AWS 리전에 있는 SageMaker 노트북을 실행하는 경우, 아래 `os.environ['AWS_DEFAULT_REGION']` 줄의 주석 처리를 해제하고 사용할 리전을 지정할 수 있습니다.

#### 특정 프로필 사용

여러 프로필로 AWS CLI를 설정한 자신의 컴퓨터에서 이 노트북을 실행하는 경우, Bedrock에 액세스할 수 있는 프로필이 기본 프로필이 아닌 경우, 아래 `os.environ['AWS_PROFILE']` 줄의 주석 처리를 해제하고 사용할 프로필을 지정할 수 있습니다.

#### 다른 Role 사용

사용자 또는 회사에서 Bedrock에 액세스하기 위해 별도의 특정 [IAM 역할](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html)을 설정한 경우, 아래 `os.environ['BEDROCK_ASSUME_ROLE']` 줄의 주석 처리를 해제하여 해당 역할을 지정할 수 있습니다. 현재 사용자 또는 역할에 해당 역할을 [assume](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html)할 수 있는 권한이 있는지 확인합니다.

#### `langchain`에 대한 참고 사항

`langchain`에서 제공하는 Bedrock 클래스는 기본적으로 Bedrock boto3 클라이언트를 생성합니다. Bedrock 구성을 커스터마이징하려면 아래 방법을 사용하여 명시적으로 Bedrock 클라이언트를 생성하고, `client=boto3_bedrock`을 사용하여 [`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) 클래스 인스턴스화 메서드에 전달할 것을 권장합니다.

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)


#### 연결 유효성 검사

`list_foundation_models()` 메서드를 사용해 클라이언트가 작동하는지 확인할 수 있으며, 이 메서드는 사용 가능한 모든 모델을 알려줍니다. 

In [ ]:
boto3_bedrock.list_foundation_models()


---

## `InvokeModel` body 및 output

Amazon Bedrock 런타임 클라이언트(`InvokeModel` API)의 `invoke_model()` 메서드는 어떤 모델을 사용하든 대부분의 텍스트 생성 및 처리 작업에 사용하는 기본 메서드입니다.

방법은 공유되지만 입력 및 출력 형식은 아래에 설명된 대로 사용되는 기초 모델에 따라 다릅니다:

### Amazon Titan Large

#### Input
```json
{   
    "inputText": "<prompt>",
    "textGenerationConfig" : { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.1,  
        "topP": 0.9
    }
}
```

#### Output

```json
{
    "inputTextTokenCount": 613,
    "results": [{
        "tokenCount": 219,
        "outputText": "<output>"
    }]
}
```

### AI21 Jurassic (Grande and Jumbo) 

#### Input

```json
{
    "prompt": "<prompt>",
    "maxTokens": 200,
    "temperature": 0.5,
    "topP": 0.5,
    "stopSequences": [],
    "countPenalty": {"scale": 0},
    "presencePenalty": {"scale": 0},
    "frequencyPenalty": {"scale": 0}
}
```

#### Output

```json
{
    "id": 1234,
    "prompt": {
        "text": "<prompt>",
        "tokens": [
            {
                "generatedToken": {
                    "token": "\u2581who\u2581is",
                    "logprob": -12.980147361755371,
                    "raw_logprob": -12.980147361755371
                },
                "topTokens": null,
                "textRange": {"start": 0, "end": 6}
            },
            //...
        ]
    },
    "completions": [
        {
            "data": {
                "text": "<output>",
                "tokens": [
                    {
                        "generatedToken": {
                            "token": "<|newline|>",
                            "logprob": 0.0,
                            "raw_logprob": -0.01293118204921484
                        },
                        "topTokens": null,
                        "textRange": {"start": 0, "end": 1}
                    },
                    //...
                ]
            },
            "finishReason": {"reason": "endoftext"}
        }
    ]
}
```

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Stability AI Stable Diffusion XL

#### Input

```json
{
    "text_prompts": [
        {"text": "this is where you place your input text"}
    ],
    "cfg_scale": 10,
    "seed": 0,
    "steps": 50
}
```

#### Output

```json
{ 
    "result": "success", 
    "artifacts": [
        {
            "seed": 123, 
            "base64": "<image in base64>",
            "finishReason": "SUCCESS"
        },
        //...
    ]
}
```

---

## 일반적인 추론 매개변수 정의

### Randomness 및 Diversity

기초 모델은 응답의 무작위성과 다양성을 제어하기 위해 다음 매개변수를 지원합니다. 
응답을 제어합니다.

**Temperature** – 대규모 언어 모델은 확률을 사용하여 단어를 순서대로 구성합니다. 주어진 다음 단어에 대해 시퀀스의 다음 단어에 대한 옵션의 확률 분포가 있습니다. Temperature를 0에 가깝게 설정하면 모델은 확률이 높은 단어를 선택하는 경향이 있습니다. Temperature를 0에서 더 멀리 설정하면 모델은 확률이 낮은 단어를 선택할 수 있습니다.

기술적인 측면에서 Temperature는 다음 토큰의 확률 밀도 함수를 조절합니다. Temperature sampling 기법을 구현합니다. 이 매개변수는 밀도 함수 곡선의 함수 곡선을 깊게 또는 평평하게 만들 수 있습니다. 값이 낮을수록 가파른 곡선으로 더 결정론적인 응답이 나타나고, 값이 높을수록 값이 높을수록 무작위 응답이 더 많은 평평한 곡선이 됩니다.

**Top K** – Temperature는 잠재적 단어의 확률 분포를 정의하고, Top K는 모델이 더 이상 단어를 선택하지 않는 컷오프 를 정의합니다. 예를 들어, K=50이면 모델은 주어진 시퀀스에서 다음에 나올 가능성이 가장 높은 단어 50개 중에서 가장 가능성이 높은 단어 50개 중에서 선택합니다. 이렇게 하면 비정상적인 단어가  단어가 다음에 선택될 확률이 줄어듭니다.
전문 용어로 Top K는 Top-K 필터링을 위해 보관할 확률이 가장 높은 어휘 토큰의 수입니다. 이는 확률 토큰의 분포를 제한하므로 모델은 가장 높은 확률의 토큰 중 하나를 선택합니다.

**Top P** – Top P는 잠재적 선택의 확률 합계를 기반으로 컷오프를 정의합니다. Top P를 1.0 미만으로 설정하면 모델은 가장 가능성이 높은 옵션을 고려하고 가능성이 낮은 옵션은 무시합니다. Top P는 Top K와 유사하지만 선택 항목 수에 제한을 두지 않습니다. 선택 항목의 확률 합계에 따라 선택 항목의 수를 제한합니다.
"I hear the hoof beats of"라는 예제 프롬프트의 경우 모델에서 "horses", "zebras" 또는 "unicorns"을 다음 단어로 제공할 수 있습니다. 상한선 없이 Temperature를 최대로 설정하는 경우 Top K 또는 Top P로 설정하면 "unicorns"과 같은 비정상적인 결과가 나올 확률이 높아집니다. Temperature를 0으로 설정하면 "말"이 나올 확률이 높아집니다. 온도를 높게 설정하고 Top K 또는 Top P를 최대로 설정하면 "horses" 또는 "zebras"이 나올 확률이 증가하고 "unicorns"이 나올 확률은 감소합니다.

### Length

다음 매개변수는 생성된 응답의 길이를 제어합니다.

**Response length** – 생성된 응답에 사용할 최소 및 최대 토큰 수를 구성합니다. 


**Length penalty** – Length penalty는 긴 응답에 불이익을 줌으로써 모델이 보다 간결하게 출력하도록 최적화합니다. Length penalty는 Response length가 최소 또는 최대 Response length에 대한 하드 컷오프이므로 Response length와는 다릅니다. 

기술적인 용어로 Length penalty는 응답이 길어지면 모델에 기하급수적으로 불이익을 주는 것입니다. 0.0 은 페널티가 없음을 의미합니다. 모델이 더 긴 시퀀스를 생성하려면 0.0보다 작은 값을 설정하고, 더 짧은 시퀀스를 생성하려면 값을 0.0보다 크게 설정하면 모델이 더 짧은 시퀀스를 생성합니다.

### Repetitions

다음 매개변수는 생성된 응답의 반복을 제어하는 데 도움이 됩니다.

**Repetition penalty (presence penalty)** – 응답에 같은 단어(토큰)가 반복되는 것을 방지합니다. 1.0은 페널티가 없습니다. 1.0보다 크면 반복이 감소합니다.

---

## Try out the models

With some theory out of the way, let's see the models in action! Run the cells below to see basic, synchronous example invocations for each model:

In [ ]:
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)


### Amazon Titan Large

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Command: Write me a blog about making strong business decisions as a leader.

Blog:
"""


In [ ]:
try:

    body = json.dumps({"inputText": prompt_data})
    modelId = "amazon.titan-tg1-large"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("results")[0].get("outputText"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### Anthropic Claude

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""


In [ ]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### AI21 Jurassic Grande

In [ ]:
body = json.dumps({"prompt": prompt_data, "maxTokens": 200})
modelId = "ai21.j2-mid-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completions")[0].get("data").get("text"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### Stability Stable Diffusion XL

In [ ]:
prompt_data = "a landscape with trees"
body = json.dumps({
    "text_prompts": [{"text": prompt_data}],
    "cfg_scale": 10,
    "seed": 20,
    "steps": 50
})
modelId = "stability.stable-diffusion-xl"
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body["result"])
    print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


**참고**: 출력은 이미지 데이터의 [base64 encoded](https://docs.python.org/3/library/base64.html) 문자열입니다. 아래 예제에서와 같이 이미지 처리 라이브러리(예: [Pillow](https://pillow.readthedocs.io/en/stable/)를 사용하여 이미지를 디코딩할 수 있습니다:

```python
import base64
import io
from PIL import Image

base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
```

In [ ]:
import base64
import io
from PIL import Image

base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
image

## 스트리밍 출력 생성

대규모 언어 모델의 경우 긴 출력 시퀀스를 생성하는 데 상당한 시간이 걸릴 수 있습니다. 지연 시간에 민감한 애플리케이션은 전체 응답을 사용할 수 있을 때까지 기다리는 대신 사용자에게 응답을 **스트리밍**하는 것을 선호할 수 있습니다.

아래 코드를 실행하여 응답 본문을 별도의 청크로 반환하는 Bedrock의 `invoke_model_with_response_stream()` 메서드를 사용하여 이를 달성하는 방법을 확인하세요.

In [ ]:
from IPython.display import clear_output, display, display_markdown, Markdown

body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-tg1-large"  # (Change this, and the request body, to try different models)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    stream = response.get('body')
    output = []

    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### Anthropic Claude

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""


In [ ]:
from IPython.display import clear_output, display, display_markdown, Markdown

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    stream = response.get('body')
    output = []

    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['completion']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


## 임베딩 생성

텍스트 임베딩을 사용하여 텍스트를 의미 있는 벡터 표현으로 변환합니다. 텍스트 본문을 입력하면 출력은 (1 x n) 벡터가 됩니다. 임베딩 벡터는 다양한 애플리케이션에 사용할 수 있습니다. 
현재 Bedrock은 텍스트 임베딩을 위해 텍스트 유사성(텍스트 본문의 의미적 유사성(의미론적 유사성 찾기) 및 텍스트 검색(예: 검색)을 지원하는 텍스트 임베딩을 제공합니다.

현재 API를 통해 임베딩 모델로 `amazon.titan-embed-text-v1`을 사용할 수 있습니다. 입력 텍스트 크기는 8192토큰이고 출력 벡터 길이는 1536입니다.

텍스트 임베딩 모델을 사용하려면 InvokeModel API 연산 또는 Python SDK를 사용합니다.
InvokeModel을 사용하여 지정된 모델에서 입력 텍스트의 벡터 표현을 검색합니다.



#### Input

```json
{
    "inputText": "<text>"
}
```

#### Output

```json
{
    "embedding": []
}
```


일부 텍스트 임베딩을 생성하는 방법을 살펴보겠습니다:

In [ ]:
prompt_data = "Amazon Bedrock supports foundation models from industry-leading providers such as \
AI21 Labs, Anthropic, Stability AI, and Amazon. Choose the model that is best suited to achieving \
your unique goals."


In [ ]:
body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-embed-text-v1"  # (Change this to try different embedding models)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    embedding = response_body.get("embedding")
    print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


## Next steps

이 노트북에서는 AWS Python SDK를 사용해 Amazon Bedrock 모델을 호출하는 몇 가지 기본 예제를 보여드렸습니다. 이제 다른 실습을 살펴보고 다양한 사용 사례와 패턴에 대해 더 자세히 알아볼 준비가 되셨습니다.